In [1]:
import fuzzingbook_utils

# Efficient Regression Testing with Mined UI Grammars

__GOAL:__ Mine a single grammar containing UI Elements, UI States, UI Interactions and Statement Coverage. Consume this grammar to efficiently generate tests.

## Roadmap

1. __DONE__ Explore an app using DroidMate _(maybe mutiple times?)_.
2. __ONGOING__ Extract a UI grammar containing only the UI elements which DM interacted with (Kotlin).
3. __DONE__ Use the fuzzingbook's GrammarCoverageFuzzer to consume the Grammar and generate input values (Python).
4. __DONE__ Extend the funzzingbook's grammar coverage tracking to consider both terminals and non-terminals (Python). 
5. _PENDING_ Associate the grammar elements with _covered statements_ (Kotlin).
6. _PENDING_ Consume expanded grammar to generatetests targeted to specific code locations (Kotlin).
7. _DONE_ Generate inputs to cover all grammar elements
8. _PENDING_ Generate inputs to cover all previously covered statements
9. _PENDING_ Generate inputs to cover specific statements

## Grammar definition:

The grammar contains two types of productions _<state>_ and _<transition>_, where:

### State (Non-Terminal <sX\>)

Represents a UI state. It contains all _UI elements_ which can be interacted with on the state, alongside their transitions. 
It is defined as 

```
<sX> ::= w1<t1>, ..., wN<tN>
```

Where:
* wX is a widget
* <tX\> is a transition

### Transition (Non-Terminal <tX\>)

Represents a UI state transition trigger by a UI element interaction. 

By design, is always associated to a UI element, but could be theoretically generalized for system events.
It is defined as

```
  <tX> ::= <s1>, ..., <sN>
```

Where _<sX\>_ is a state. 
A transition contains only UI state productions. 
It also contains _all UI states_ which can be reached by this transition.

### Widget/UI Element (Terminal wX)

A UI element represents a UI element interaction in the app. It is defined as 

```
wX ≡ widgetX.actionY[.dataZ]
```

Where:
* _widgetX_ is the unique UI element ID
* _actionY_ is the type of action performed
* _dataZ_ is the value used in the action, such as text input, if any. This parameter is optional.

This notation can simultaneously encode:
* UI elements
* Different action types on the same UI elements (click, long click, tick, etc, input text)
* Different input payloads (searched value)

### Terminate condition

An exploration can be over at any time. Thus, all _State_ productions can also be $\epsilon$.



# Concrete Example

## Notation

* State: Sequential numbering
* Widget: Sequential numbering (later mapped to UID)
* Action types: Click and Long Click
* Transition: <state.widget>. 
For example `s6.w7.Click` stands for: the widget `w7.Click` in the state `s6`.
This allows the same widget and interaction to have different transitions in different states.


## Grammar

In [2]:
simple_ui_grammar = {
'<empty>'             : 		[''],
'<s1.w0.Click>'       : 		['<empty>', '<s2>'],
'<s1>'                : 		['w0.Click <s1.w0.Click>'],
'<s2.w1.Click>'       : 		['<empty>', '<s3>'],
'<s2.w3.Click>'       : 		['<empty>', '<s4>'],
'<s2>'                : 		['w1.Click <s2.w1.Click>', 'w3.Click <s2.w3.Click>'],
'<s3.w2.Click>'       : 		['<empty>', '<s2>'],
'<s3>'                : 		['w2.Click <s3.w2.Click>'],
'<s4.w4.Click>'       : 		['<empty>', '<s4>'],
'<s4.w5.LongClick>'   : 		['<empty>', '<s5>'],
'<s4>'                : 		['w4.Click <s4.w4.Click>', 'w5.LongClick <s4.w5.LongClick>'],
'<s5.w6.Click>'       : 		['<empty>', '<s6>'],
'<s5>'                : 		['w6.Click <s5.w6.Click>'],
'<s6.w7.Click>'       : 		['<empty>', '<s6>'],
'<s6.w8.Click>'       : 		['<empty>', '<s6>'],
'<s6>'                : 		['w7.Click <s6.w7.Click>', 'w8.Click <s6.w8.Click>'],
'<start>'             : 		['<empty>', '<s1>']
}

# Fuzzing

## Using expansion coverage

With fuzzingbook's default implementation

In [3]:
from GrammarCoverageFuzzer import GrammarCoverageFuzzer

In [4]:
fuzzer = GrammarCoverageFuzzer(simple_ui_grammar, min_nonterminals=1)
max_exp = fuzzer.max_expansion_coverage(max_depth=len(simple_ui_grammar))
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    print(fuzzer.fuzz())


w0.Click w1.Click w2.Click 
w0.Click 
w0.Click w3.Click 
w0.Click w3.Click w4.Click 
w0.Click w1.Click 
w0.Click w3.Click w5.LongClick 
w0.Click w3.Click w5.LongClick w6.Click w8.Click 
w0.Click w3.Click w4.Click w5.LongClick w6.Click 
w0.Click w1.Click w2.Click w3.Click w5.LongClick w6.Click w7.Click 
w0.Click w3.Click w5.LongClick w6.Click w7.Click w8.Click w8.Click w7.Click 


## Using Term Coverage (Terminals and non-Terminals)

Extension of fuzzingbook's implementation.

Greedy algorithm.

In [5]:
import re
from Grammars import START_SYMBOL, RE_NONTERMINAL, nonterminals
from GrammarCoverageFuzzer import GrammarCoverageFuzzer

### Extended Coverage measurement

Number of times each symbol has been seem (necessary as all states can finish with $\epsilon$)

In [6]:
class TerminalCoverageGrammar(GrammarCoverageFuzzer):
    def add_coverage(self, symbol, new_children):
        """Coverage computation: number of times each symbol has been seen"""
        if self.log and symbol not in self.covered_expansions.keys():
            print("Now covered:", symbol)

        if symbol not in self.covered_expansions.keys():
            self.covered_expansions[symbol] = 0

        curr_val = self.covered_expansions[symbol]
        self.covered_expansions[symbol] = curr_val + 1
    def reset_coverage(self):
        self.covered_expansions = dict()

    def expansion_coverage(self):
        return self.covered_expansions.keys()

### Obtain set of existing symbols

Compute all existing symbols (terminals and non terminals) in the grammar:

In [7]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def all_symbols_grammar(self):
        symbols = { START_SYMBOL }

        for symbol in self.grammar.keys():
            for expansion in self.grammar[symbol]:
                for expansion_segment in list(filter(lambda x: x != "", re.split(RE_NONTERMINAL, expansion))):
                    symbols.add(expansion_segment)

        return symbols


### Redefine non-covered elements

Expansions with non-covered elements must use the new data type:

In [8]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def _uncovered_children(self, possible_children):
        # Prefer uncovered expansions
        uncovered_children = []
        index_map = []
        for idx, child_list in enumerate(possible_children):
            for child_tuple in child_list:
                (child, grand_child) = child_tuple
                if child not in self.expansion_coverage() and child_list not in uncovered_children:
                    uncovered_children.append(child_list)
                    index_map.append(idx)

        return index_map, uncovered_children

### Determine minimally covered elements

The same UI elements may be available in multiple states. Thus we should prioretize elements which have been seen the least when all elements have already been covered.

In [9]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def _minimally_expanded(self, possible_children):
        symbol_count = dict()
        for idx, child_list in enumerate(possible_children):
            for child_tuple in child_list:
                (child, grand_child) = child_tuple

                assert(child in self.covered_expansions.keys())
                symbol_count[child] = self.covered_expansions[child]

        min_count = min(symbol_count.values())

        # Prefer least covered expansions
        uncovered_children = []
        index_map = []
        for idx, child_list in enumerate(possible_children):
            for child_tuple in child_list:
                (child, grand_child) = child_tuple
                if symbol_count[child] == min_count and child_list not in uncovered_children:
                    uncovered_children.append(child_list)
                    index_map.append(idx)

        return index_map, uncovered_children

### Add terminals to coverage

Add terminals to the list of covered grammar elements

In [10]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def process_chosen_children(self, chosen_children, expansion):
        """Add terminals to coverage."""
        for expansion_segment in list(filter(lambda x: x != "", re.split(RE_NONTERMINAL, expansion))):
            if expansion_segment not in nonterminals(expansion):
                self.add_coverage(expansion_segment, chosen_children)
        return chosen_children


### Choose node expansion

Update node expansion rules to guide expansion towards least explored elements

In [11]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def choose_node_expansion(self, node, possible_children):
        index_map, uncovered_children = self._uncovered_children(possible_children)

        if len(uncovered_children) == 0:
            # Search for least explored
            index_map, min_covered_children = self._minimally_expanded(possible_children)
            # All expansions covered the same amount of times - use superclass method
            index = self.choose_covered_node_expansion(node, min_covered_children)

            return index_map[index]

        # Select from uncovered nodes
        index = self.choose_uncovered_node_expansion(node, uncovered_children)

        return index_map[index]

### Running the Fuzzer

In [12]:
fuzzer = TerminalCoverageGrammar(simple_ui_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    print(fuzzer.fuzz())

w0.Click w1.Click 

w0.Click w3.Click w5.LongClick w6.Click w7.Click w8.Click w8.Click 
w0.Click w3.Click w4.Click w5.LongClick w6.Click 
w0.Click w1.Click w2.Click 


# Larger example (XKCD - 20 Actions)

In [13]:
xkcd_grammar = {
'<empty>'             : 		[''],
'<s01.w00.ClickEvent>' : 		['<empty>', '<s02>'],
'<s01.w03.ClickEvent>' : 		['<empty>', '<s04>'],
'<s01.w07.ClickEvent>' : 		['<empty>', '<s07>'],
'<s01>'               : 		['w00.ClickEvent <s01.w00.ClickEvent>', 'w02.LongClickEvent <start>', 'w03.ClickEvent <s01.w03.ClickEvent>', 'w06.LongClickEvent <start>', 'w07.ClickEvent <s01.w07.ClickEvent>'],
'<s02.w01.Tick>'      : 		['<empty>', '<s03>'],
'<s02>'               : 		['w01.Tick <s02.w01.Tick>'],
'<s03.PressBack>'     : 		['<empty>', '<s01>', '<s07>'],
'<s03>'               : 		['<s03.PressBack>'],
'<s04.w04.ClickEvent>' : 		['<empty>', '<s05>'],
'<s04>'               : 		['w04.ClickEvent <s04.w04.ClickEvent>'],
'<s05.w05.ClickEvent>' : 		['<empty>', '<s06>'],
'<s05>'               : 		['w05.ClickEvent <s05.w05.ClickEvent>'],
'<s06>'               : 		['<start>'],
'<s07.w00.ClickEvent>' : 		['<empty>', '<s09>'],
'<s07.w02.ClickEvent>' : 		['<empty>', '<s10>'],
'<s07.w06.ClickEvent>' : 		['<empty>', '<s08>'],
'<s07>'               : 		['<empty>', 'w00.ClickEvent <s07.w00.ClickEvent>', 'w02.ClickEvent <s07.w02.ClickEvent>', 'w03.LongClickEvent <s01.w07.ClickEvent>', 'w06.ClickEvent <s07.w06.ClickEvent>', 'w07.ClickEvent <s01.w07.ClickEvent>'],
'<s08>'               : 		['w08.ClickEvent <s01.w07.ClickEvent>'],
'<s09>'               : 		['w09.ClickEvent <s02.w01.Tick>'],
'<s10>'               : 		['w10.Click <s01.w07.ClickEvent>'],
'<start>'             : 		['<empty>', '<s01>']
}

In [14]:
fuzzer = TerminalCoverageGrammar(xkcd_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
reached = set()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    inp = fuzzer.fuzz()
    reached.add(inp)
    
for x in reached:
    print(x)


w02.LongClickEvent 
w07.ClickEvent w00.ClickEvent w09.ClickEvent 
w00.ClickEvent w01.Tick w03.LongClickEvent w02.ClickEvent w10.Click w06.ClickEvent w08.ClickEvent w06.ClickEvent w08.ClickEvent 
w06.LongClickEvent 
w03.ClickEvent w04.ClickEvent w05.ClickEvent 


# Another Example (aMetro - 20 Actions)

In [15]:
ametro_grammar = {
    '<ClickEvent(s01.w00)>' : 		['<empty>', '<s02>'],
    '<ClickEvent(s01.w03)>' : 		['<empty>', '<s03>'],
    '<ClickEvent(s02.w08)>' : 		['<empty>', '<s05>'],
    '<ClickEvent(s02.w12)>' : 		['<empty>', '<s07>'],
    '<ClickEvent(s03.w04)>' : 		['<empty>', '<s04>'],
    '<ClickEvent(s05.w09)>' : 		['<empty>', '<s06>'],
    '<ClickEvent(s07.w13)>' : 		['<empty>', '<s08>'],
    '<ClickEvent(s08.w14)>' : 		['<empty>', '<s09>'],
    '<ClickEvent(s09.w15)>' : 		['<empty>', '<s10>'],
    '<empty>'             : 		[''],
    '<s01>'               : 		['ClickEvent(w00) <ClickEvent(s01.w00)>', 'ClickEvent(w03) <ClickEvent(s01.w03)>', 'LongClickEvent(w02) <start>'],
    '<s02>'               : 		['ClickEvent(w01) <start>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w12) <ClickEvent(s02.w12)>'],
    '<s03>'               : 		['ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w06) <start>', 'ClickEvent(w07) <ClickEvent(s01.w00)>'],
    '<s04>'               : 		['ClickEvent(w01) <ClickEvent(s01.w03)>', 'LongClickEvent(w05) <ClickEvent(s03.w04)>'],
    '<s05>'               : 		['ClickEvent(w09) <ClickEvent(s05.w09)>'],
    '<s06>'               : 		['ClickEvent(w11) <ClickEvent(s01.w00)>', 'LongClickEvent(w10) <ClickEvent(s05.w09)>'],
    '<s07>'               : 		['ClickEvent(w13) <ClickEvent(s07.w13)>'],
    '<s08>'               : 		['ClickEvent(w14) <ClickEvent(s08.w14)>'],
    '<s09>'               : 		['ClickEvent(w15) <ClickEvent(s09.w15)>'],
    '<s10>'               : 		['<empty>', 'ClickEvent(w16) <ClickEvent(s09.w15)>'],
    '<start>'             : 		['<empty>', '<s01>']
}

In [16]:
fuzzer = TerminalCoverageGrammar(ametro_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
reached = set()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    inp = fuzzer.fuzz()
    reached.add(inp)
    
for x in reached:
    print(x)


LongClickEvent(w02) ClickEvent(w00) ClickEvent(w08) ClickEvent(w09) LongClickEvent(w10) ClickEvent(w11) ClickEvent(w12) ClickEvent(w13) ClickEvent(w14) ClickEvent(w15) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) 
ClickEvent(w03) ClickEvent(w07) ClickEvent(w12) ClickEvent(w13) ClickEvent(w14) ClickEvent(w15) ClickEvent(w16) ClickEvent(w16) 
ClickEvent(w00) ClickEvent(w08) ClickEvent(w09) ClickEvent(w11) ClickEvent(w08) ClickEvent(w09) LongClickEvent(w10) LongClickEvent(w10) ClickEvent(w11) ClickEvent(w01) ClickEvent(w03) ClickEvent(w04) LongClickEvent(w05) LongClickEvent(w05) LongClickEvent(w05) LongClickEvent(w05) ClickEvent(w01) ClickEvent(w07) ClickEvent(w12) ClickEvent(w13) ClickEvent(w14) ClickEvent(w15) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) 
ClickEvent(w03) ClickEvent(w06) 
ClickEvent(w03) 
LongClickEvent(w02) 
LongClickEvent(w02) ClickEvent(w00) ClickEvent(w01) 
ClickEvent(w00) ClickEvent(w01) 


# aMetro - 100 actions

In [17]:
test_grammar = {
'<Click(s16.w27)>'    : 		['<empty>', '<s27>'],
'<Click(s19.w27)>'    : 		['<empty>', '<s20>'],
'<Click(s25.w27)>'    : 		['<empty>', '<s26>'],
'<ClickEvent(s01.w00)>' : 		['<empty>', '<s02>', '<s16>'],
'<ClickEvent(s01.w03)>' : 		['<empty>', '<s03>'],
'<ClickEvent(s02.w08)>' : 		['<empty>', '<s05>'],
'<ClickEvent(s02.w10)>' : 		['<empty>', '<s07>'],
'<ClickEvent(s02.w12)>' : 		['<empty>', '<s07>', '<s13>'],
'<ClickEvent(s03.w04)>' : 		['<empty>', '<s04>'],
'<ClickEvent(s03.w07)>' : 		['<empty>', '<s02>'],
'<ClickEvent(s04.w01)>' : 		['<empty>', '<s03>', '<s27>'],
'<ClickEvent(s05.w09)>' : 		['<empty>', '<s06>'],
'<ClickEvent(s07.w02)>' : 		['<empty>', '<s08>'],
'<ClickEvent(s08.w13)>' : 		['<empty>', '<s09>'],
'<ClickEvent(s09.w14)>' : 		['<empty>', '<s10>'],
'<ClickEvent(s09.w16)>' : 		['<empty>', '<s11>'],
'<ClickEvent(s11.w17)>' : 		['<empty>', '<s12>'],
'<ClickEvent(s12.w18)>' : 		['<empty>', '<s13>'],
'<ClickEvent(s12.w25)>' : 		['<empty>', '<s18>'],
'<ClickEvent(s13.w19)>' : 		['<empty>', '<s14>'],
'<ClickEvent(s13.w26)>' : 		['<empty>', '<s28>'],
'<ClickEvent(s13.w35)>' : 		['<empty>', '<s23>'],
'<ClickEvent(s14.w20)>' : 		['<empty>', '<s15>'],
'<ClickEvent(s15.w21)>' : 		['<empty>', '<s00>'],
'<ClickEvent(s18.w14)>' : 		['<empty>', '<s24>'],
'<ClickEvent(s18.w26)>' : 		['<empty>', '<s19>'],
'<ClickEvent(s21.w02)>' : 		['<empty>', '<s30>'],
'<ClickEvent(s21.w03)>' : 		['<empty>', '<s22>', '<s27>'],
'<ClickEvent(s21.w28)>' : 		['<empty>', '<s21>'],
'<ClickEvent(s22.w30)>' : 		['<empty>', '<s22>'],
'<ClickEvent(s24.w36)>' : 		['<empty>', '<s25>'],
'<ClickEvent(s26.w11)>' : 		['<empty>', '<s16>'],
'<ClickEvent(s28.w24)>' : 		['<empty>', '<s29>'],
'<LongClickEvent(s16.w22)>' : 		['<empty>', '<s17>'],
'<PressBack(s00)>'    : 		['<s01>'],
'<PressBack(s20)>'    : 		['<s21>'],
'<TextInsert(s30.w38,lixxdwxhjctsalrmgb)>' : 		['<empty>', '<s31>'],
'<empty>'             : 		[''],
'<s00>'               : 		['<PressBack(s00)>'],
'<s01>'               : 		['ClickEvent(w00) <ClickEvent(s01.w00)>', 'ClickEvent(w03) <ClickEvent(s01.w03)>', 'LongClickEvent(w02) <start>'],
'<s02>'               : 		['ClickEvent(w01) <start>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w10) <ClickEvent(s02.w10)>', 'ClickEvent(w12) <ClickEvent(s02.w12)>', 'LongClickEvent(w02) <ClickEvent(s03.w07)>', 'LongClickEvent(w10) <ClickEvent(s03.w07)>'],
'<s03>'               : 		['ClickEvent(w00) <ClickEvent(s03.w07)>', 'ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w06) <start>', 'ClickEvent(w07) <ClickEvent(s03.w07)>'],
'<s04>'               : 		['ClickEvent(w01) <ClickEvent(s04.w01)>', 'LongClickEvent(w05) <ClickEvent(s03.w04)>'],
'<s05>'               : 		['ClickEvent(w09) <ClickEvent(s05.w09)>', 'ClickEvent(w34) <ClickEvent(s01.w00)>'],
'<s06>'               : 		['ClickEvent(w11) <ClickEvent(s03.w07)>', 'ClickEvent(w12) <ClickEvent(s12.w18)>', 'LongClickEvent(w10) <ClickEvent(s05.w09)>', 'LongClickEvent(w23) <ClickEvent(s05.w09)>'],
'<s07>'               : 		['ClickEvent(w01) <ClickEvent(s03.w07)>', 'ClickEvent(w02) <ClickEvent(s07.w02)>'],
'<s08>'               : 		['ClickEvent(w13) <ClickEvent(s08.w13)>'],
'<s09>'               : 		['ClickEvent(w14) <ClickEvent(s09.w14)>', 'ClickEvent(w16) <ClickEvent(s09.w16)>'],
'<s10>'               : 		['ClickEvent(w15) <ClickEvent(s08.w13)>'],
'<s11>'               : 		['ClickEvent(w17) <ClickEvent(s11.w17)>'],
'<s12>'               : 		['ClickEvent(w18) <ClickEvent(s12.w18)>', 'ClickEvent(w25) <ClickEvent(s12.w25)>'],
'<s13>'               : 		['ClickEvent(w19) <ClickEvent(s13.w19)>', 'ClickEvent(w24) <ClickEvent(s11.w17)>', 'ClickEvent(w26) <ClickEvent(s13.w26)>', 'ClickEvent(w35) <ClickEvent(s13.w35)>'],
'<s14>'               : 		['ClickEvent(w20) <ClickEvent(s14.w20)>'],
'<s15>'               : 		['ClickEvent(w21) <ClickEvent(s15.w21)>'],
'<s16>'               : 		['Click(w27) <Click(s16.w27)>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w10) <ClickEvent(s12.w18)>', 'LongClickEvent(w22) <LongClickEvent(s16.w22)>'],
'<s17>'               : 		['ClickEvent(w23) <ClickEvent(s03.w07)>'],
'<s18>'               : 		['ClickEvent(w14) <ClickEvent(s18.w14)>', 'ClickEvent(w26) <ClickEvent(s18.w26)>'],
'<s19>'               : 		['Click(w27) <Click(s19.w27)>'],
'<s20>'               : 		['<PressBack(s20)>'],
'<s21>'               : 		['ClickEvent(w02) <ClickEvent(s21.w02)>', 'ClickEvent(w03) <ClickEvent(s21.w03)>', 'ClickEvent(w28) <ClickEvent(s21.w28)>', 'LongClickEvent(w02) <ClickEvent(s21.w28)>'],
'<s22>'               : 		['ClickEvent(w29) <ClickEvent(s21.w28)>', 'ClickEvent(w30) <ClickEvent(s22.w30)>', 'ClickEvent(w31) <ClickEvent(s21.w28)>', 'ClickEvent(w33) <ClickEvent(s22.w30)>', 'Tick(w32) <ClickEvent(s22.w30)>'],
'<s23>'               : 		['ClickEvent(w21) <ClickEvent(s12.w25)>'],
'<s24>'               : 		['ClickEvent(w36) <ClickEvent(s24.w36)>'],
'<s25>'               : 		['Click(w27) <Click(s25.w27)>'],
'<s26>'               : 		['ClickEvent(w11) <ClickEvent(s26.w11)>'],
'<s27>'               : 		['<empty>', 'ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w07) <ClickEvent(s26.w11)>', 'ClickEvent(w29) <ClickEvent(s21.w28)>', 'ClickEvent(w31) <ClickEvent(s21.w28)>'],
'<s28>'               : 		['ClickEvent(w24) <ClickEvent(s28.w24)>'],
'<s29>'               : 		['ClickEvent(w37) <ClickEvent(s21.w28)>'],
'<s30>'               : 		['ClickEvent(w17) <ClickEvent(s21.w28)>', 'ClickEvent(w28) <ClickEvent(s21.w02)>', 'TextInsert(w38,bexyp) <TextInsert(s30.w38,lixxdwxhjctsalrmgb)>', 'TextInsert(w38,lixxdwxhjctsalrmgb) <TextInsert(s30.w38,lixxdwxhjctsalrmgb)>'],
'<s31>'               : 		['ClickEvent(w17) <ClickEvent(s21.w28)>', 'ClickEvent(w39) <ClickEvent(s21.w02)>'],
'<start>'             : 		['<empty>', '<s01>'],
}

In [18]:
fuzzer = TerminalCoverageGrammar(test_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
reached = set()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    inp = fuzzer.fuzz()
    reached.add(inp)
    
for x in reached:
    print(x)


ClickEvent(w03) ClickEvent(w07) ClickEvent(w12) ClickEvent(w02) ClickEvent(w13) ClickEvent(w14) ClickEvent(w15) ClickEvent(w16) ClickEvent(w17) ClickEvent(w18) ClickEvent(w26) ClickEvent(w24) ClickEvent(w37) ClickEvent(w03) ClickEvent(w04) LongClickEvent(w05) LongClickEvent(w05) ClickEvent(w01) ClickEvent(w29) 
ClickEvent(w00) ClickEvent(w08) ClickEvent(w09) LongClickEvent(w23) LongClickEvent(w23) ClickEvent(w12) ClickEvent(w19) ClickEvent(w20) ClickEvent(w21) ClickEvent(w03) ClickEvent(w07) ClickEvent(w12) ClickEvent(w01) ClickEvent(w12) ClickEvent(w26) ClickEvent(w24) ClickEvent(w37) ClickEvent(w28) 
ClickEvent(w00) ClickEvent(w10) ClickEvent(w24) ClickEvent(w18) ClickEvent(w35) ClickEvent(w21) ClickEvent(w26) Click(w27) ClickEvent(w02) TextInsert(w38,bexyp) ClickEvent(w39) TextInsert(w38,lixxdwxhjctsalrmgb) ClickEvent(w17) ClickEvent(w03) ClickEvent(w30) Tick(w32) Tick(w32) ClickEvent(w33) ClickEvent(w30) ClickEvent(w31) ClickEvent(w28) LongClickEvent(w02) ClickEvent(w28) 
ClickEve

## Get all terminals from the grammar

In [19]:
list(filter(lambda x: not "<" in x, fuzzer.all_symbols_grammar()))

['Tick(w32) ',
 'ClickEvent(w18) ',
 'ClickEvent(w21) ',
 'ClickEvent(w01) ',
 'ClickEvent(w06) ',
 'ClickEvent(w24) ',
 'ClickEvent(w12) ',
 'Click(w27) ',
 'ClickEvent(w10) ',
 'ClickEvent(w36) ',
 'ClickEvent(w16) ',
 'TextInsert(w38,bexyp) ',
 'ClickEvent(w29) ',
 'LongClickEvent(w05) ',
 'ClickEvent(w34) ',
 'ClickEvent(w20) ',
 'ClickEvent(w04) ',
 'ClickEvent(w03) ',
 'LongClickEvent(w02) ',
 'LongClickEvent(w23) ',
 'ClickEvent(w28) ',
 'TextInsert(w38,lixxdwxhjctsalrmgb) ',
 'ClickEvent(w31) ',
 'ClickEvent(w09) ',
 'ClickEvent(w25) ',
 'ClickEvent(w35) ',
 'LongClickEvent(w10) ',
 'ClickEvent(w30) ',
 'ClickEvent(w11) ',
 'ClickEvent(w14) ',
 'ClickEvent(w08) ',
 'ClickEvent(w02) ',
 'LongClickEvent(w22) ',
 'ClickEvent(w07) ',
 'ClickEvent(w13) ',
 'ClickEvent(w37) ',
 'ClickEvent(w19) ',
 'ClickEvent(w00) ',
 'ClickEvent(w23) ',
 'ClickEvent(w33) ',
 'ClickEvent(w39) ',
 'ClickEvent(w17) ',
 'ClickEvent(w26) ',
 'ClickEvent(w15) ']

# aMetro 1000

In [20]:
test_grammar = {
'<Click(s09.w22)>'    : 		['<empty>', '<s10>'],
'<ClickEvent(s01.w00)>' : 		['<empty>', '<s02>', '<s09>', '<s29>', '<s71>'],
'<ClickEvent(s01.w02)>' : 		['<empty>', '<s11>'],
'<ClickEvent(s01.w03)>' : 		['<empty>', '<s03>'],
'<ClickEvent(s02.w01)>' : 		['<empty>', '<s01>', '<s03>', '<s11>'],
'<ClickEvent(s02.w02)>' : 		['<empty>', '<s48>'],
'<ClickEvent(s02.w08)>' : 		['<empty>', '<s05>'],
'<ClickEvent(s02.w12)>' : 		['<empty>', '<s07>', '<s13>'],
'<ClickEvent(s03.w00)>' : 		['<empty>', '<s02>', '<s09>', '<s29>', '<s43>'],
'<ClickEvent(s03.w02)>' : 		['<empty>', '<s34>'],
'<ClickEvent(s03.w04)>' : 		['<empty>', '<s04>'],
'<ClickEvent(s03.w07)>' : 		['<empty>', '<s02>', '<s09>', '<s29>', '<s43>', '<s71>'],
'<ClickEvent(s04.w01)>' : 		['<empty>', '<s03>', '<s21>', '<s30>', '<s34>', '<s65>'],
'<ClickEvent(s05.w09)>' : 		['<empty>', '<s06>', '<s24>', '<s45>', '<s51>'],
'<ClickEvent(s06.w11)>' : 		['<empty>', '<s02>'],
'<ClickEvent(s07.w01)>' : 		['<empty>', '<s02>', '<s39>'],
'<ClickEvent(s07.w13)>' : 		['<empty>', '<s08>'],
'<ClickEvent(s07.w20)>' : 		['<empty>', '<s14>'],
'<ClickEvent(s07.w23)>' : 		['<empty>', '<s15>'],
'<ClickEvent(s07.w26)>' : 		['<empty>', '<s08>', '<s14>'],
'<ClickEvent(s07.w28)>' : 		['<empty>', '<s15>', '<s25>'],
'<ClickEvent(s07.w33)>' : 		['<empty>', '<s14>', '<s25>'],
'<ClickEvent(s08.w14)>' : 		['<empty>', '<s00>'],
'<ClickEvent(s08.w27)>' : 		['<empty>', '<s38>'],
'<ClickEvent(s09.w01)>' : 		['<empty>', '<s01>', '<s03>', '<s11>', '<s20>'],
'<ClickEvent(s09.w02)>' : 		['<empty>', '<s35>'],
'<ClickEvent(s09.w10)>' : 		['<empty>', '<s07>'],
'<ClickEvent(s09.w15)>' : 		['<empty>', '<s10>', '<s20>'],
'<ClickEvent(s11.w00)>' : 		['<empty>', '<s02>', '<s09>'],
'<ClickEvent(s13.w02)>' : 		['<empty>', '<s53>'],
'<ClickEvent(s14.w20)>' : 		['<empty>', '<s27>'],
'<ClickEvent(s14.w21)>' : 		['<empty>', '<s09>', '<s27>'],
'<ClickEvent(s14.w25)>' : 		['<empty>', '<s45>'],
'<ClickEvent(s14.w26)>' : 		['<empty>', '<s15>', '<s27>'],
'<ClickEvent(s15.w23)>' : 		['<empty>', '<s58>'],
'<ClickEvent(s15.w24)>' : 		['<empty>', '<s16>'],
'<ClickEvent(s15.w25)>' : 		['<empty>', '<s33>'],
'<ClickEvent(s15.w28)>' : 		['<empty>', '<s24>'],
'<ClickEvent(s16.w24)>' : 		['<empty>', '<s42>'],
'<ClickEvent(s16.w25)>' : 		['<empty>', '<s17>'],
'<ClickEvent(s16.w28)>' : 		['<empty>', '<s43>'],
'<ClickEvent(s17.w26)>' : 		['<empty>', '<s18>'],
'<ClickEvent(s18.w27)>' : 		['<empty>', '<s19>'],
'<ClickEvent(s19.w28)>' : 		['<empty>', '<s09>'],
'<ClickEvent(s19.w40)>' : 		['<empty>', '<s50>'],
'<ClickEvent(s20.w02)>' : 		['<empty>', '<s22>'],
'<ClickEvent(s20.w03)>' : 		['<empty>', '<s21>', '<s31>', '<s32>', '<s65>'],
'<ClickEvent(s20.w29)>' : 		['<empty>', '<s20>'],
'<ClickEvent(s21.w02)>' : 		['<empty>', '<s30>'],
'<ClickEvent(s21.w07)>' : 		['<empty>', '<s09>', '<s29>', '<s43>', '<s64>', '<s71>'],
'<ClickEvent(s21.w29)>' : 		['<empty>', '<s21>'],
'<ClickEvent(s24.w01)>' : 		['<empty>', '<s03>', '<s12>'],
'<ClickEvent(s25.w26)>' : 		['<empty>', '<s62>'],
'<ClickEvent(s25.w27)>' : 		['<empty>', '<s28>'],
'<ClickEvent(s27.w14)>' : 		['<empty>', '<s40>'],
'<ClickEvent(s27.w26)>' : 		['<empty>', '<s49>'],
'<ClickEvent(s27.w28)>' : 		['<empty>', '<s25>'],
'<ClickEvent(s28.w13)>' : 		['<empty>', '<s29>'],
'<ClickEvent(s28.w34)>' : 		['<empty>', '<s71>'],
'<ClickEvent(s29.w02)>' : 		['<empty>', '<s59>'],
'<ClickEvent(s35.w08)>' : 		['<empty>', '<s37>'],
'<ClickEvent(s37.w09)>' : 		['<empty>', '<s35>', '<s59>'],
'<ClickEvent(s37.w39)>' : 		['<empty>', '<s38>', '<s67>'],
'<ClickEvent(s39.w18)>' : 		['<empty>', '<s09>', '<s43>'],
'<ClickEvent(s40.w24)>' : 		['<empty>', '<s41>'],
'<ClickEvent(s43.w01)>' : 		['<empty>', '<s01>', '<s20>'],
'<ClickEvent(s43.w02)>' : 		['<empty>', '<s47>'],
'<ClickEvent(s43.w36)>' : 		['<empty>', '<s20>', '<s23>'],
'<ClickEvent(s45.w41)>' : 		['<empty>', '<s46>'],
'<ClickEvent(s50.w34)>' : 		['<empty>', '<s51>'],
'<ClickEvent(s53.w21)>' : 		['<empty>', '<s55>'],
'<ClickEvent(s53.w33)>' : 		['<empty>', '<s54>'],
'<ClickEvent(s53.w34)>' : 		['<empty>', '<s70>'],
'<ClickEvent(s55.w33)>' : 		['<empty>', '<s57>'],
'<ClickEvent(s55.w34)>' : 		['<empty>', '<s56>'],
'<ClickEvent(s63.w32)>' : 		['<empty>', '<s64>'],
'<ClickEvent(s72.w44)>' : 		['<empty>', '<s71>', '<s72>'],
'<LongClickEvent(s06.w10)>' : 		['<empty>', '<s06>'],
'<LongClickEvent(s09.w15)>' : 		['<empty>', '<s26>'],
'<LongClickEvent(s13.w02)>' : 		['<empty>', '<s13>'],
'<LongClickEvent(s29.w36)>' : 		['<empty>', '<s52>'],
'<LongClickEvent(s35.w15)>' : 		['<empty>', '<s36>'],
'<LongClickEvent(s43.w41)>' : 		['<empty>', '<s44>'],
'<LongClickEvent(s59.w36)>' : 		['<empty>', '<s60>'],
'<LongClickEvent(s62.w42)>' : 		['<empty>', '<s63>'],
'<LongClickEvent(s64.w36)>' : 		['<empty>', '<s66>'],
'<LongClickEvent(s71.w44)>' : 		['<empty>', '<s72>'],
'<PressBack(s00)>'    : 		['<s01>'],
'<PressBack(s10)>'    : 		['<s00>', '<s10>', '<s20>', '<s22>'],
'<PressBack(s42)>'    : 		['<s01>', '<s16>', '<s20>'],
'<TextInsert(s11.w16,rpv)>' : 		['<empty>', '<s12>'],
'<TextInsert(s22.w16,vka)>' : 		['<empty>', '<s23>'],
'<TextInsert(s30.w16,skyeyftvldfdgvqvxcbiet)>' : 		['<empty>', '<s61>'],
'<TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>' : 		['<empty>', '<s39>'],
'<TextInsert(s38.w16,wabd)>' : 		['<empty>', '<s68>'],
'<TextInsert(s48.w16,mecrw)>' : 		['<empty>', '<s69>'],
'<TextInsert(s53.w16,vejm)>' : 		['<empty>', '<s73>'],
'<empty>'             : 		[''],
'<s00>'               : 		['<empty>', 'PressBack(s00) <PressBack(s00)>'],
'<s01>'               : 		['ClickEvent(w00) <ClickEvent(s01.w00)>', 'ClickEvent(w02) <ClickEvent(s01.w02)>', 'ClickEvent(w03) <ClickEvent(s01.w03)>', 'LongClickEvent(w02) <start>'],
'<s02>'               : 		['ClickEvent(w01) <ClickEvent(s02.w01)>', 'ClickEvent(w02) <ClickEvent(s02.w02)>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w10) <ClickEvent(s02.w12)>', 'ClickEvent(w12) <ClickEvent(s02.w12)>', 'LongClickEvent(w02) <ClickEvent(s06.w11)>', 'LongClickEvent(w10) <ClickEvent(s06.w11)>'],
'<s03>'               : 		['ClickEvent(w00) <ClickEvent(s03.w00)>', 'ClickEvent(w02) <ClickEvent(s03.w02)>', 'ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w06) <start>', 'ClickEvent(w07) <ClickEvent(s03.w07)>', 'LongClickEvent(w02) <ClickEvent(s01.w03)>'],
'<s04>'               : 		['ClickEvent(w01) <ClickEvent(s04.w01)>', 'ClickEvent(w05) <ClickEvent(s03.w04)>', 'LongClickEvent(w05) <ClickEvent(s03.w04)>'],
'<s05>'               : 		['ClickEvent(w09) <ClickEvent(s05.w09)>', 'ClickEvent(w19) <ClickEvent(s01.w00)>'],
'<s06>'               : 		['ClickEvent(w01) <start>', 'ClickEvent(w11) <ClickEvent(s06.w11)>', 'ClickEvent(w12) <ClickEvent(s09.w10)>', 'ClickEvent(w32) <ClickEvent(s06.w11)>', 'LongClickEvent(w10) <LongClickEvent(s06.w10)>'],
'<s07>'               : 		['<s66>', 'ClickEvent(w01) <ClickEvent(s07.w01)>', 'ClickEvent(w02) <ClickEvent(s13.w02)>', 'ClickEvent(w13) <ClickEvent(s07.w13)>', 'ClickEvent(w20) <ClickEvent(s07.w20)>', 'ClickEvent(w21) <ClickEvent(s07.w20)>', 'ClickEvent(w23) <ClickEvent(s07.w23)>', 'ClickEvent(w25) <ClickEvent(s07.w20)>', 'ClickEvent(w26) <ClickEvent(s07.w26)>', 'ClickEvent(w27) <ClickEvent(s07.w20)>', 'ClickEvent(w28) <ClickEvent(s07.w28)>', 'ClickEvent(w33) <ClickEvent(s07.w33)>', 'ClickEvent(w34) <ClickEvent(s07.w20)>', 'ClickEvent(w35) <ClickEvent(s07.w20)>', 'LongClickEvent(w02) <ClickEvent(s09.w10)>'],
'<s08>'               : 		['ClickEvent(w14) <ClickEvent(s08.w14)>', 'ClickEvent(w24) <ClickEvent(s40.w24)>', 'ClickEvent(w27) <ClickEvent(s08.w27)>'],
'<s09>'               : 		['Click(w22) <Click(s09.w22)>', 'ClickEvent(w01) <ClickEvent(s09.w01)>', 'ClickEvent(w02) <ClickEvent(s09.w02)>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w10) <ClickEvent(s09.w10)>', 'ClickEvent(w15) <ClickEvent(s09.w15)>', 'LongClickEvent(w02) <ClickEvent(s19.w28)>', 'LongClickEvent(w10) <ClickEvent(s19.w28)>', 'LongClickEvent(w15) <LongClickEvent(s09.w15)>'],
'<s10>'               : 		['<empty>', 'PressBack(s10) <PressBack(s10)>'],
'<s11>'               : 		['ClickEvent(w00) <ClickEvent(s11.w00)>', 'ClickEvent(w18) <start>', 'TextInsert(w16,atdqfumfhlbjb) <TextInsert(s11.w16,rpv)>', 'TextInsert(w16,cna) <TextInsert(s11.w16,rpv)>', 'TextInsert(w16,mbiqgkbmknrhjrbqeoz) <TextInsert(s11.w16,rpv)>', 'TextInsert(w16,rpv) <TextInsert(s11.w16,rpv)>', 'TextInsert(w16,ycisxhjxcylfw) <TextInsert(s11.w16,rpv)>'],
'<s12>'               : 		['ClickEvent(w00) <ClickEvent(s11.w00)>', 'ClickEvent(w17) <ClickEvent(s01.w02)>', 'ClickEvent(w18) <start>'],
'<s13>'               : 		['ClickEvent(w01) <ClickEvent(s06.w11)>', 'ClickEvent(w02) <ClickEvent(s13.w02)>', 'LongClickEvent(w02) <LongClickEvent(s13.w02)>'],
'<s14>'               : 		['ClickEvent(w01) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>', 'ClickEvent(w02) <ClickEvent(s13.w02)>', 'ClickEvent(w20) <ClickEvent(s14.w20)>', 'ClickEvent(w21) <ClickEvent(s14.w21)>', 'ClickEvent(w23) <ClickEvent(s07.w28)>', 'ClickEvent(w25) <ClickEvent(s14.w25)>', 'ClickEvent(w26) <ClickEvent(s14.w26)>', 'ClickEvent(w27) <ClickEvent(s14.w20)>', 'ClickEvent(w28) <ClickEvent(s14.w20)>', 'ClickEvent(w33) <ClickEvent(s27.w28)>', 'ClickEvent(w34) <ClickEvent(s15.w25)>', 'ClickEvent(w35) <ClickEvent(s14.w20)>'],
'<s15>'               : 		['ClickEvent(w14) <ClickEvent(s16.w28)>', 'ClickEvent(w23) <ClickEvent(s15.w23)>', 'ClickEvent(w24) <ClickEvent(s15.w24)>', 'ClickEvent(w25) <ClickEvent(s15.w25)>', 'ClickEvent(w26) <ClickEvent(s28.w13)>', 'ClickEvent(w28) <ClickEvent(s15.w28)>'],
'<s16>'               : 		['ClickEvent(w13) <ClickEvent(s16.w24)>', 'ClickEvent(w24) <ClickEvent(s16.w24)>', 'ClickEvent(w25) <ClickEvent(s16.w25)>', 'ClickEvent(w28) <ClickEvent(s16.w28)>', 'ClickEvent(w40) <ClickEvent(s15.w24)>'],
'<s17>'               : 		['ClickEvent(w26) <ClickEvent(s17.w26)>'],
'<s18>'               : 		['ClickEvent(w27) <ClickEvent(s18.w27)>'],
'<s19>'               : 		['ClickEvent(w28) <ClickEvent(s19.w28)>', 'ClickEvent(w40) <ClickEvent(s19.w40)>'],
'<s20>'               : 		['<empty>', 'ClickEvent(w02) <ClickEvent(s20.w02)>', 'ClickEvent(w03) <ClickEvent(s20.w03)>', 'ClickEvent(w29) <ClickEvent(s20.w29)>', 'LongClickEvent(w02) <ClickEvent(s20.w29)>'],
'<s21>'               : 		['<empty>', 'ClickEvent(w02) <ClickEvent(s21.w02)>', 'ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w06) <ClickEvent(s20.w29)>', 'ClickEvent(w07) <ClickEvent(s21.w07)>', 'ClickEvent(w29) <ClickEvent(s21.w29)>', 'ClickEvent(w30) <ClickEvent(s20.w29)>', 'ClickEvent(w31) <ClickEvent(s20.w29)>', 'LongClickEvent(w02) <ClickEvent(s21.w29)>'],
'<s22>'               : 		['<empty>', 'ClickEvent(w18) <ClickEvent(s20.w29)>', 'ClickEvent(w29) <ClickEvent(s20.w02)>', 'TextInsert(w16,avwwvqipxauwwktfkikh) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,bcqgebyiq) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,bediwos) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,cfpdjqfkglthzqmnqczs) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,degyrtkt) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,dpjcvl) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,fhdnwbruxzbvoualqdqlrq) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,fllyppaymgdgfwvvvpj) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,fowirhhruatgshxbxtid) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,havdbqxptyyvx) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,hvkjlg) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,jxvqohkuovp) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,kxwvkd) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,mgurjcnetdvu) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,nbbptjtizbjdfmukyxxbm) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,ppkbyxahzbgizp) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,seowcevrstbbeyhgje) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,sgcvmaeyeofsydyku) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,uvarwdfkpistrd) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,vka) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,wdopjawpuosgt) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,xvtampzmroiijok) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,yixuj) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,yrk) <TextInsert(s22.w16,vka)>', 'TextInsert(w16,zbnuojsimcb) <TextInsert(s22.w16,vka)>'],
'<s23>'               : 		['ClickEvent(w17) <ClickEvent(s20.w02)>', 'ClickEvent(w18) <ClickEvent(s20.w29)>', 'ClickEvent(w29) <TextInsert(s22.w16,vka)>'],
'<s24>'               : 		['ClickEvent(w01) <ClickEvent(s24.w01)>', 'ClickEvent(w10) <ClickEvent(s09.w10)>', 'ClickEvent(w11) <ClickEvent(s19.w28)>', 'ClickEvent(w32) <ClickEvent(s19.w28)>', 'LongClickEvent(w02) <ClickEvent(s15.w28)>', 'LongClickEvent(w10) <ClickEvent(s15.w28)>', 'LongClickEvent(w32) <ClickEvent(s15.w28)>'],
'<s25>'               : 		['ClickEvent(w01) <ClickEvent(s19.w28)>', 'ClickEvent(w13) <ClickEvent(s16.w24)>', 'ClickEvent(w14) <ClickEvent(s15.w24)>', 'ClickEvent(w20) <ClickEvent(s15.w25)>', 'ClickEvent(w21) <ClickEvent(s14.w20)>', 'ClickEvent(w23) <ClickEvent(s16.w28)>', 'ClickEvent(w25) <ClickEvent(s15.w25)>', 'ClickEvent(w26) <ClickEvent(s25.w26)>', 'ClickEvent(w27) <ClickEvent(s25.w27)>', 'ClickEvent(w28) <ClickEvent(s25.w27)>', 'ClickEvent(w33) <ClickEvent(s19.w28)>', 'ClickEvent(w34) <ClickEvent(s19.w28)>', 'ClickEvent(w35) <ClickEvent(s15.w25)>'],
'<s26>'               : 		['ClickEvent(w11) <ClickEvent(s19.w28)>'],
'<s27>'               : 		['ClickEvent(w14) <ClickEvent(s27.w14)>', 'ClickEvent(w20) <ClickEvent(s27.w28)>', 'ClickEvent(w21) <ClickEvent(s15.w25)>', 'ClickEvent(w23) <ClickEvent(s27.w28)>', 'ClickEvent(w25) <ClickEvent(s27.w28)>', 'ClickEvent(w26) <ClickEvent(s27.w26)>', 'ClickEvent(w27) <ClickEvent(s07.w20)>', 'ClickEvent(w28) <ClickEvent(s27.w28)>', 'ClickEvent(w33) <ClickEvent(s27.w28)>', 'ClickEvent(w34) <ClickEvent(s27.w28)>', 'ClickEvent(w35) <ClickEvent(s14.w20)>'],
'<s28>'               : 		['ClickEvent(w13) <ClickEvent(s28.w13)>', 'ClickEvent(w14) <ClickEvent(s07.w13)>', 'ClickEvent(w34) <ClickEvent(s28.w34)>'],
'<s29>'               : 		['ClickEvent(w01) <ClickEvent(s02.w01)>', 'ClickEvent(w02) <ClickEvent(s29.w02)>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w10) <ClickEvent(s09.w10)>', 'ClickEvent(w15) <ClickEvent(s20.w29)>', 'ClickEvent(w36) <ClickEvent(s09.w15)>', 'LongClickEvent(w02) <ClickEvent(s28.w13)>', 'LongClickEvent(w10) <ClickEvent(s28.w13)>', 'LongClickEvent(w36) <LongClickEvent(s29.w36)>'],
'<s30>'               : 		['ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w07) <ClickEvent(s16.w28)>', 'ClickEvent(w18) <ClickEvent(s21.w29)>', 'ClickEvent(w29) <ClickEvent(s21.w02)>', 'ClickEvent(w30) <ClickEvent(s20.w02)>', 'ClickEvent(w31) <ClickEvent(s20.w02)>', 'TextInsert(w16,skyeyftvldfdgvqvxcbiet) <TextInsert(s30.w16,skyeyftvldfdgvqvxcbiet)>'],
'<s31>'               : 		['ClickEvent(w37) <ClickEvent(s20.w29)>'],
'<s32>'               : 		['ClickEvent(w38) <ClickEvent(s20.w29)>'],
'<s33>'               : 		['ClickEvent(w14) <ClickEvent(s28.w13)>', 'ClickEvent(w21) <ClickEvent(s15.w25)>', 'ClickEvent(w23) <ClickEvent(s27.w28)>', 'ClickEvent(w26) <ClickEvent(s25.w27)>', 'ClickEvent(w27) <ClickEvent(s09.w02)>', 'ClickEvent(w28) <ClickEvent(s19.w28)>', 'ClickEvent(w33) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>', 'ClickEvent(w34) <ClickEvent(s14.w20)>'],
'<s34>'               : 		['ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w07) <ClickEvent(s19.w28)>'],
'<s35>'               : 		['ClickEvent(w08) <ClickEvent(s35.w08)>', 'ClickEvent(w18) <ClickEvent(s19.w28)>', 'LongClickEvent(w15) <LongClickEvent(s35.w15)>', 'TextInsert(w16,ctlulpliywrbgdwbxvt) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>', 'TextInsert(w16,cyfisshurmlicnuhl) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>', 'TextInsert(w16,hmmzh) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>', 'TextInsert(w16,udusdawgvftkgkszmu) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>', 'TextInsert(w16,wozybij) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>'],
'<s36>'               : 		['ClickEvent(w11) <ClickEvent(s09.w02)>'],
'<s37>'               : 		['ClickEvent(w09) <ClickEvent(s37.w09)>', 'ClickEvent(w19) <ClickEvent(s09.w10)>', 'ClickEvent(w39) <ClickEvent(s37.w39)>'],
'<s38>'               : 		['ClickEvent(w11) <ClickEvent(s09.w02)>', 'ClickEvent(w18) <ClickEvent(s15.w28)>', 'ClickEvent(w32) <ClickEvent(s09.w02)>', 'TextInsert(w16,wabd) <TextInsert(s38.w16,wabd)>'],
'<s39>'               : 		['<empty>', 'ClickEvent(w08) <ClickEvent(s35.w08)>', 'ClickEvent(w17) <ClickEvent(s09.w02)>', 'ClickEvent(w18) <ClickEvent(s39.w18)>'],
'<s40>'               : 		['ClickEvent(w24) <ClickEvent(s40.w24)>'],
'<s41>'               : 		['ClickEvent(w14) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>', 'ClickEvent(w35) <ClickEvent(s15.w24)>'],
'<s42>'               : 		['PressBack(s42) <PressBack(s42)>'],
'<s43>'               : 		['ClickEvent(w01) <ClickEvent(s43.w01)>', 'ClickEvent(w02) <ClickEvent(s43.w02)>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w10) <ClickEvent(s09.w10)>', 'ClickEvent(w36) <ClickEvent(s43.w36)>', 'ClickEvent(w41) <ClickEvent(s20.w29)>', 'LongClickEvent(w10) <ClickEvent(s16.w28)>', 'LongClickEvent(w36) <LongClickEvent(s43.w41)>', 'LongClickEvent(w41) <LongClickEvent(s43.w41)>'],
'<s44>'               : 		['ClickEvent(w01) <start>', 'ClickEvent(w11) <ClickEvent(s16.w28)>', 'ClickEvent(w36) <LongClickEvent(s43.w41)>', 'ClickEvent(w41) <ClickEvent(s16.w28)>', 'LongClickEvent(w10) <LongClickEvent(s43.w41)>', 'LongClickEvent(w32) <LongClickEvent(s43.w41)>'],
'<s45>'               : 		['ClickEvent(w11) <ClickEvent(s16.w28)>', 'ClickEvent(w36) <ClickEvent(s45.w41)>', 'ClickEvent(w41) <ClickEvent(s45.w41)>', 'LongClickEvent(w32) <ClickEvent(s14.w25)>'],
'<s46>'               : 		['ClickEvent(w01) <ClickEvent(s20.w29)>', 'ClickEvent(w11) <ClickEvent(s16.w28)>', 'LongClickEvent(w10) <ClickEvent(s45.w41)>', 'LongClickEvent(w32) <ClickEvent(s45.w41)>', 'LongClickEvent(w36) <ClickEvent(s45.w41)>', 'LongClickEvent(w41) <ClickEvent(s45.w41)>'],
'<s47>'               : 		['ClickEvent(w36) <ClickEvent(s20.w02)>', 'ClickEvent(w41) <ClickEvent(s20.w29)>'],
'<s48>'               : 		['ClickEvent(w08) <ClickEvent(s35.w08)>', 'ClickEvent(w12) <ClickEvent(s09.w10)>', 'TextInsert(w16,mecrw) <TextInsert(s48.w16,mecrw)>'],
'<s49>'               : 		['ClickEvent(w24) <ClickEvent(s18.w27)>'],
'<s50>'               : 		['ClickEvent(w34) <ClickEvent(s50.w34)>', 'ClickEvent(w40) <ClickEvent(s19.w40)>'],
'<s51>'               : 		['ClickEvent(w01) <ClickEvent(s01.w03)>', 'ClickEvent(w10) <ClickEvent(s09.w10)>', 'ClickEvent(w11) <ClickEvent(s28.w13)>', 'LongClickEvent(w32) <ClickEvent(s50.w34)>', 'LongClickEvent(w36) <ClickEvent(s50.w34)>'],
'<s52>'               : 		['ClickEvent(w01) <start>', 'ClickEvent(w11) <ClickEvent(s28.w13)>', 'ClickEvent(w36) <ClickEvent(s28.w13)>', 'LongClickEvent(w10) <LongClickEvent(s29.w36)>', 'LongClickEvent(w32) <LongClickEvent(s29.w36)>'],
'<s53>'               : 		['ClickEvent(w18) <ClickEvent(s09.w10)>', 'ClickEvent(w20) <ClickEvent(s53.w21)>', 'ClickEvent(w21) <ClickEvent(s53.w21)>', 'ClickEvent(w25) <ClickEvent(s53.w21)>', 'ClickEvent(w33) <ClickEvent(s53.w33)>', 'ClickEvent(w34) <ClickEvent(s53.w34)>', 'ClickEvent(w35) <ClickEvent(s53.w21)>', 'TextInsert(w16,vejm) <TextInsert(s53.w16,vejm)>'],
'<s54>'               : 		['ClickEvent(w20) <ClickEvent(s13.w02)>', 'ClickEvent(w33) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>'],
'<s55>'               : 		['ClickEvent(w20) <ClickEvent(s13.w02)>', 'ClickEvent(w21) <ClickEvent(s13.w02)>', 'ClickEvent(w25) <ClickEvent(s28.w13)>', 'ClickEvent(w33) <ClickEvent(s55.w33)>', 'ClickEvent(w34) <ClickEvent(s55.w34)>'],
'<s56>'               : 		['ClickEvent(w27) <ClickEvent(s55.w34)>', 'ClickEvent(w35) <ClickEvent(s53.w21)>'],
'<s57>'               : 		['ClickEvent(w20) <ClickEvent(s19.w28)>'],
'<s58>'               : 		['PressBack(s58) <PressBack(s00)>'],
'<s59>'               : 		['ClickEvent(w08) <ClickEvent(s35.w08)>', 'ClickEvent(w18) <ClickEvent(s28.w13)>', 'LongClickEvent(w36) <LongClickEvent(s59.w36)>', 'TextInsert(w16,dmsigrqwy) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>'],
'<s60>'               : 		['ClickEvent(w11) <ClickEvent(s29.w02)>', 'LongClickEvent(w32) <LongClickEvent(s59.w36)>'],
'<s61>'               : 		['ClickEvent(w30) <TextInsert(s22.w16,vka)>'],
'<s62>'               : 		['LongClickEvent(w42) <LongClickEvent(s62.w42)>', 'LongClickEvent(w44) <LongClickEvent(s62.w42)>'],
'<s63>'               : 		['ClickEvent(w11) <ClickEvent(s25.w26)>', 'ClickEvent(w32) <ClickEvent(s63.w32)>', 'ClickEvent(w41) <LongClickEvent(s62.w42)>', 'ClickEvent(w42) <LongClickEvent(s62.w42)>', 'ClickEvent(w43) <LongClickEvent(s62.w42)>', 'ClickEvent(w44) <LongClickEvent(s62.w42)>'],
'<s64>'               : 		['ClickEvent(w41) <ClickEvent(s20.w29)>', 'ClickEvent(w43) <ClickEvent(s21.w29)>', 'ClickEvent(w44) <ClickEvent(s20.w29)>', 'LongClickEvent(w36) <LongClickEvent(s64.w36)>'],
'<s65>'               : 		['ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w06) <ClickEvent(s20.w29)>', 'ClickEvent(w07) <ClickEvent(s63.w32)>', 'ClickEvent(w30) <ClickEvent(s20.w29)>', 'ClickEvent(w31) <ClickEvent(s20.w29)>', 'ClickEvent(w37) <ClickEvent(s20.w29)>', 'ClickEvent(w38) <ClickEvent(s20.w29)>', 'ClickEvent(w45) <ClickEvent(s20.w29)>', 'ClickEvent(w46) <ClickEvent(s20.w29)>', 'ClickEvent(w47) <ClickEvent(s20.w29)>'],
'<s66>'               : 		['<empty>'],
'<s67>'               : 		['ClickEvent(w12) <ClickEvent(s09.w10)>'],
'<s68>'               : 		['ClickEvent(w11) <TextInsert(s35.w16,ctlulpliywrbgdwbxvt)>', 'LongClickEvent(w32) <TextInsert(s38.w16,wabd)>'],
'<s69>'               : 		['ClickEvent(w12) <ClickEvent(s09.w10)>'],
'<s70>'               : 		['ClickEvent(w21) <ClickEvent(s53.w33)>'],
'<s71>'               : 		['ClickEvent(w01) <ClickEvent(s01.w03)>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w41) <ClickEvent(s20.w29)>', 'LongClickEvent(w02) <ClickEvent(s28.w34)>', 'LongClickEvent(w10) <ClickEvent(s28.w34)>', 'LongClickEvent(w36) <LongClickEvent(s71.w44)>', 'LongClickEvent(w41) <LongClickEvent(s71.w44)>', 'LongClickEvent(w44) <LongClickEvent(s71.w44)>'],
'<s72>'               : 		['ClickEvent(w01) <start>', 'ClickEvent(w11) <ClickEvent(s28.w34)>', 'ClickEvent(w32) <ClickEvent(s19.w28)>', 'ClickEvent(w36) <LongClickEvent(s71.w44)>', 'ClickEvent(w41) <ClickEvent(s72.w44)>', 'ClickEvent(w44) <ClickEvent(s72.w44)>', 'LongClickEvent(w02) <LongClickEvent(s71.w44)>', 'LongClickEvent(w10) <LongClickEvent(s71.w44)>', 'LongClickEvent(w32) <LongClickEvent(s71.w44)>'],
'<s73>'               : 		['ClickEvent(w17) <ClickEvent(s13.w02)>'],
'<start>'             : 		['<empty>', '<s01>'],
}

In [32]:
fuzzer = TerminalCoverageGrammar(test_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
reached = set()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    inp = fuzzer.fuzz()
    reached.add(inp)
    
for x in reached:
    print(x)


ClickEvent(w00) LongClickEvent(w44) ClickEvent(w44) ClickEvent(w11) ClickEvent(w08) ClickEvent(w09) LongClickEvent(w32) LongClickEvent(w32) LongClickEvent(w32) LongClickEvent(w32) ClickEvent(w36) LongClickEvent(w10) LongClickEvent(w32) LongClickEvent(w36) ClickEvent(w11) ClickEvent(w02) ClickEvent(w41) 
ClickEvent(w03) ClickEvent(w07) LongClickEvent(w02) LongClickEvent(w44) ClickEvent(w44) ClickEvent(w41) ClickEvent(w32) ClickEvent(w15) PressBack(s10) PressBack(s00) ClickEvent(w02) TextInsert(w16,rpv) ClickEvent(w00) LongClickEvent(w15) ClickEvent(w11) ClickEvent(w01) TextInsert(w16,atdqfumfhlbjb) ClickEvent(w00) ClickEvent(w10) LongClickEvent(w02) ClickEvent(w02) ClickEvent(w34) ClickEvent(w21) ClickEvent(w20) TextInsert(w16,vejm) ClickEvent(w17) TextInsert(w16,vejm) ClickEvent(w17) ClickEvent(w34) ClickEvent(w21) ClickEvent(w20) ClickEvent(w25) ClickEvent(w33) ClickEvent(w20) ClickEvent(w01) ClickEvent(w00) ClickEvent(w01) TextInsert(w16,mbiqgkbmknrhjrbqeoz) ClickEvent(w17) TextInse